In [134]:
import pandas as pd
import numpy as np
import datetime as dt
import math as mt
import IPython
import IPython.display

# Calculator

In [120]:
# 1~6, 9~12
low_pressure_fee = [
    [0,200,910,88.3],
    [201,400,1600, 182.9],
    [401,float('inf'),7300, 275.6]
]
# 7~8
low_pressure_fee_summer = [
    [0,300,910,88.3],
    [301,450,1600,182.9],
    [451,float('inf'),7300,275.6]
]
# 1~6, 9~12
high_pressure_fee = [
    [0,200,730,73.3],
    [201,400,1260,142.3],
    [400,float('inf'),6060,210.6]
]
# 7~8
high_pressure_fee_summer = [
    [0,300,730,73.3],
    [301,450,1260,142.3],
    [451,float('inf'),6060,210.6]
]

general_fee = {
    "저압": [6160, 100.7, 60.2, 87.3],
    "고압 A": [
        [7170, 110.9, 66.9, 98.6],
        [8230, 106.9, 62.6, 93.3]
    ],
    "고압 B": [
        [7170, 108.8, 65.8, 95.6],
        [8230, 103.5, 60.5, 90.3]
    ]
}
env_fee = 5.3
fuel_fee = 3
VAT = 0.1
FUND = 0.037

class Household:
    def __init__(self,
                name, kwh, contract, contract_name):
        self.name = name
        self.kwh = kwh
        self.contract = contract
        self.contract_name = contract_name
        
    def set_bill(self,
                public_fee):
        self.self_fee = self.elec_bill_vat_fund
        self.public_fee = public_fee
        self.bill = self.self_fee + self.public_fee
        
    @property
    def basic(self):
        fee = None
        for _ in self.contract:
            if _[0] <= self.kwh and _[1] >= self.kwh:
                fee = _[2]
                break
        return fee
    
    @property
    def elec_rate(self):
        kwh = self.kwh
        fee = 0
        for _ in self.contract:
            if kwh <= _[1]:
                fee += (kwh * _[3])
                break
            else:
                kwh -= _[1]
                fee += (_[1] * _[3])
        
        return mt.floor(fee)
        
    @property
    def guarantee(self):
        if self.kwh <= 200:
            if self.contract_name == "종합계약":
                return 4000
            elif self.contract_name == "단일계약":
                return 2500
        else:
            return 0
        
    @property
    def elec_bill(self):
        bill = self.basic + self.elec_rate\
            - self.guarantee + self.env - self.fuel
        
        if bill < 1000:
            return 1000
        else:
            return bill
    
class Public:
    def __init__(self,
                kwh,
                charge_applied,
                contract):
        # 공용설비사용량
        self.kwh = kwh
        # 요금적용전력
        self.charge_applied = charge_applied
        self.contract = contract
        
    @property
    def basic(self):
        return mt.floor(self.charge_applied * self.contract[0])
    
    @property
    def elec_rate(self):
        return mt.floor(self.kwh * self.contract[1])
    
    @property
    def elec_bill(self):
        bill = self.basic + self.elec_rate + self.env - self.fuel
        
        if bill < 1000:
            return 1000
        else:
            return bill
    
@property
def env(self):
    return mt.floor(self.kwh * env_fee)
@property
def fuel(self):
    return mt.floor(self.kwh * fuel_fee)
@property
def vat(self):
    return round(self.elec_bill * 0.1)
@property
def fund(self):
    return mt.floor(self.elec_bill * 0.037 * 0.1) * 10
@property
def elec_bill_vat_fund(self):
    return mt.floor((self.elec_bill + self.vat + self.fund) * 0.1) * 10

Household.env = env
Household.fuel = fuel
Household.vat = vat
Household.fund = fund
Household.elec_bill_vat_fund = elec_bill_vat_fund

Public.env = env
Public.fuel = fuel
Public.vat = vat
Public.fund = fund
Public.elec_bill_vat_fund = elec_bill_vat_fund
    
class ManagementOffice:
    def __init__(self, 
                 month, peaks, households, APT, # datas
                 contract,
                 general_fee_info=None):
        print("[관리사무소] Init.")
        
        self.peaks = peaks
        self.APT = APT
        
        self.contract = contract
        self.select_fee(month, contract, general_fee_info)
        
        if contract == "종합계약":
            # 가구 객체화
            # - 가구별 지정된 계약에 요금들이 계산되도록 Property 구성해놨음.
            print("[관리사무소] 가구 객체화")
            self.households = list()
            for idx in households.index:
                self.households.append(
                    Household(
                        name=households.iloc[idx]['name'],
                        kwh=households.iloc[idx]['usage (kWh)'],
                        contract=self.fee[0],
                        contract_name=contract
                    )
                )

            # 공용설비사용요금
            print("[관리사무소] 공용설비사용요금 계산")
            households_kwh = sum([_.kwh for _ in self.households])
            public_kwh = APT - households_kwh
            max_peak = peak_df['peak (kW)'].max()
            charge_applied = max_peak * (public_kwh / APT)
            
            self.public = Public(
                kwh=public_kwh,
                charge_applied=charge_applied,
                contract=self.fee[1]
            )

            # 가구별 청구서 셋팅
            public_fee = round(self.public.elec_bill_vat_fund / len(self.households) / 10) * 10
            
            print("public fee", public_fee)
            for household in self.households:
                household.set_bill(
                    public_fee=public_fee
                )

            # 아파트 청구서 셋팅
            bill = 0
            for household in self.households:
                bill += household.bill
            self.bill = bill
        elif contract == "단일계약":
            # 전체를 하나의 가구 본 상태에서 계약대로 계산식 진행
            # 후에 * len(household) 를 통해 아파트 전체 요금 통지서 확보
            num_household = len(households)
            mean_kwh = APT / num_household

            apart = Household(
                name="아파트",
                kwh=mean_kwh,
                contract=self.fee,
                contract_name=contract
            )
            basic = apart.basic * num_household
            elec_rate = apart.elec_rate * num_household
            env = apart.env * num_household
            fuel = apart.fuel * num_household
            
            elec_bill = basic + elec_rate + env - fuel
            self.bill = mt.floor((elec_bill \
                + round(elec_bill * 0.1) \
                + mt.floor(elec_bill * 0.037 * 0.1) * 10) * 0.1) * 10
            
            # 가구 객체화
            # - 가구별 지정된 계약에 요금들이 계산되도록 Property 구성해놨음.
            print("[관리사무소] 가구 객체화")
            self.households = list()
            for idx in households.index:
                self.households.append(
                    Household(
                        name=households.iloc[idx]['name'],
                        kwh=households.iloc[idx]['usage (kWh)'],
                        contract=self.fee,
                        contract_name=contract
                    )
                )
                
            # 공공설비사용요금
            households_bill = sum(
                [_.elec_bill_vat_fund for _ in self.households]
            )
            print("가구 청구 비용 : {}".format(format(households_bill)))
            self.public_bill = self.bill - households_bill
            
            
            # 가구별 청구서 셋팅
            public_fee = round(self.public_bill / len(self.households))
            for household in self.households:
                household.set_bill(
                    public_fee=public_fee
                )

    # 요금제 셋팅 메서드
    def select_fee(self, month, contract, general_fee_info):
            print("[관리사무소] 계약 정보 셋팅")

            if contract == "종합계약":
                household_fee = None
                public_fee = None
                if (month >= 1 and month <= 6) or\
                    (month >= 9 and month <= 12):
                    household_fee = low_pressure_fee
                else:
                    household_fee = low_pressure_fee_summer

                if general_fee_info == None:
                    raise Exception("종합계약은 일반용 전력 정보를 포함해야 합니다.\n")

                tmp_general_fee = None
                if general_fee_info == "저압":
                    tmp_general_fee = general_fee[general_fee_info]
                elif len(general_fee_info) == 2:
                    tmp_general_fee = general_fee[general_fee_info[0]]
                    tmp_general_fee = tmp_general_fee[general_fee_info[1]]
                else: 
                    raise Exception("일반용 전력 설정이 올바르지 않습니다\n"\
                                + "저압, [고압 A, 0 or 1], [고압 B, 0 or 1]")

                if month >= 6 and month <= 8:
                    public_fee = [tmp_general_fee[0], tmp_general_fee[1]]
                elif (month >= 3 and month <= 5) or\
                      (month >= 9 and month <= 10):
                    public_fee = [tmp_general_fee[0], tmp_general_fee[2]]
                elif (month >= 1 and month <= 2) or\
                      (month >= 11 and month <= 12):
                    public_fee = [tmp_general_fee[0], tmp_general_fee[3]]
                self.fee = [household_fee, public_fee]
                
            elif contract == "단일계약":
                if (month >= 1 and month <= 6) or\
                    (month >= 9 and month <= 12):
                    self.fee = high_pressure_fee
                else:
                    self.fee = high_pressure_fee_summer

            print("[관리사무소] 계약 정보 셋팅 완료\n")
            print(self.fee)        
            print("")

# Data Processing

In [5]:
xlsx = pd.read_excel("datas/datas.xlsx", header=None, skiprows=2, engine="openpyxl")

In [91]:
date_df = xlsx[3:][xlsx.columns[1:6]].copy()
household_df = xlsx[xlsx.columns[7:]]

date_list = [dt.datetime(
    date_df.loc[_][1],
    date_df.loc[_][2],
    date_df.loc[_][3],
    date_df.loc[_][4],
    date_df.loc[_][5]
) for _ in date_df.index]

datas_df = pd.DataFrame(columns=['date'])
datas_df['date'] = date_list

for col in household_df:
    household_name = "{}-{}-{}".format(
        household_df[col][0],
        household_df[col][1],
        household_df[col][2]
    )
    datas_df[household_name] = household_df[col][3:].to_list()
    
datas_df = datas_df.replace("-", 0)

sum_df = pd.DataFrame(columns=['date', 'kWh', 'kW'])
sum_df['date'] = date_list
sum_df['kWh'] = [round (_) for _ in datas_df[datas_df.columns.difference(['date'])].sum(axis=1).to_list()]
sum_df['kW'] = (sum_df['kWh'] / 0.25).to_list()

peak_df = pd.DataFrame(columns=['month', 'peak (kW)'])
for month in range(1,13):
    peak_df = peak_df.append({
        "month": str(month),
        "peak (kW)": sum_df[sum_df['date'].dt.month == month]['kW'].max()
    }, ignore_index=True)

month_usage_df = pd.DataFrame(columns=['month'])
month_usage_df['month'] = [_ for _ in range(1,13)]

for name in datas_df[datas_df.columns.difference(['date'])]:
    self_household_df = datas_df[['date',name]].copy()
    
    month_usage_df[name] = [
        round(self_household_df[self_household_df['date'].dt.month == month][name].sum())
        for month in range(1,13)
    ]  
    
month_usage_df

,month,아파트1-101-1002,아파트1-101-104,아파트1-101-1102,아파트1-101-1202,아파트1-101-1302,아파트1-101-1401,아파트1-101-1403,아파트1-101-1502,아파트1-101-1504,...,아파트4-104-101,아파트4-104-1301,아파트4-104-1501,아파트4-104-301,아파트4-105-102,아파트4-105-103,아파트4-105-1102,아파트4-105-1104,아파트4-105-601,아파트4-105-803
0,1,253,689,183,239,455,343,380,219,389,...,335,363,181,216,270,518,230,279,161,156
1,2,251,481,177,218,397,297,339,197,348,...,309,339,146,184,267,344,208,236,167,150
2,3,255,424,203,222,378,303,349,202,357,...,313,372,152,236,268,417,212,272,166,155
3,4,252,355,180,230,368,278,298,188,346,...,266,346,149,210,268,421,208,243,156,148
4,5,230,357,258,203,414,270,418,156,320,...,305,256,151,222,287,349,185,284,165,180
5,6,212,363,272,207,411,275,398,149,265,...,240,257,145,212,353,413,212,292,153,148
6,7,326,423,416,204,597,350,477,210,374,...,492,469,179,294,566,574,362,459,181,227
7,8,431,454,450,207,701,411,505,236,385,...,512,394,177,283,559,562,245,388,199,224
8,9,207,334,198,188,445,299,369,153,261,...,274,242,146,233,328,371,207,278,182,181
9,10,219,390,135,212,439,281,346,175,296,...,280,254,165,232,278,357,227,294,147,181


In [92]:
peak_df

,month,peak (kW)
0,1,224.0
1,2,224.0
2,3,212.0
3,4,208.0
4,5,152.0
5,6,200.0
6,7,356.0
7,8,356.0
8,9,204.0
9,10,192.0


# Calculator Test

## 1. Calculator 에서 처리 가능한 DataFrame으로 월별 데이터 생성

In [93]:
analysis_df = month_usage_df.set_index("month")
analysis_df

# 1. 월별 사용량 데이터 파싱
month = 1

month_datas_df = pd.DataFrame(columns=["name","usage (kWh)"])
for idx in analysis_df.iloc[month].index:
    household_name = idx
    household_kWh = analysis_df.iloc[month][idx]
    
    month_datas_df = month_datas_df.append({
        "name": household_name,
        "usage (kWh)": household_kWh
    }, ignore_index=True)
    
month_datas_df

,name,usage (kWh)
0,아파트1-101-1002,251
1,아파트1-101-104,481
2,아파트1-101-1102,177
3,아파트1-101-1202,218
4,아파트1-101-1302,397
...,...,...
393,아파트4-105-103,344
394,아파트4-105-1102,208
395,아파트4-105-1104,236
396,아파트4-105-601,167


## 2. Setting APT ( 공공설비사용량 테스트 )

In [126]:
# Thinking
# 세대부 전기는 정해져 있는데,
# 공용부 전기는 정해져 있지가 않아서 입력되는 percentage에 따라, 변화하도록

# 다음과 같은 공식을 사용할 수 있다.
# n -> 공용부가 전체 APT에서 차지할 percentage
# APT : households_kWh = 100 : (100 - n)
# APT : public_kWh = 100 : n

# 이에 따라,
# APT = (households_kWh * 100) / (100 - n)
# public_kwh = APT - households_kwh
PUBLIC_PERCENTAGE = 30

households_kWh = sum(month_datas_df['usage (kWh)'].values)
APT = round((households_kWh * 100) / (100 - PUBLIC_PERCENTAGE))
public_kWh = round(APT - households_kWh)

print("전체 사용량 : {}kWh\n".format(APT) +
     "세대부 사용량 : {}kWh\n".format(households_kWh) +
     "공용부 사용량 : {}kWh".format(public_kWh))

전체 사용량 : 144149kWh
세대부 사용량 : 100904kWh
공용부 사용량 : 43245kWh


## 3. set bill ( calculator 사용 )

In [127]:
# 종합계약
calc = ManagementOffice(
    month=month,
    peaks=peak_df, 
    households=month_datas_df,
    APT=APT,
    contract="종합계약",
    general_fee_info=['고압 A', 1]
)

# 단일계약
single_calc = ManagementOffice(
    month=month,
    peaks=peak_df, 
    households=month_datas_df,
    APT=APT,
    contract="단일계약"
)

[관리사무소] Init.
[관리사무소] 계약 정보 셋팅
[관리사무소] 계약 정보 셋팅 완료

[[[0, 200, 910, 88.3], [201, 400, 1600, 182.9], [401, inf, 7300, 275.6]], [8230, 93.3]]

[관리사무소] 가구 객체화
[관리사무소] 공용설비사용요금 계산
public fee 14320
[관리사무소] Init.
[관리사무소] 계약 정보 셋팅
[관리사무소] 계약 정보 셋팅 완료

[[0, 200, 730, 73.3], [201, 400, 1260, 142.3], [400, inf, 6060, 210.6]]

[관리사무소] 가구 객체화
가구 청구 비용 : 11093590


## 4. Analysis

In [128]:
print("입력값 정보------------------------------")
print("공용부 Percentage : {}%".format(PUBLIC_PERCENTAGE))
print("--------------------------------------\n")

print("{}월 요금 정보---------------------------".format(month))
print("종합계약 아파트 전기 사용요금 : {} 원".format(format(calc.bill, ",")))
print("단일계약 아파트 전기 사용요금 : {} 원".format(format(single_calc.bill, ",")))
print("--------------------------------------\n")

# 각 계약 손해가구, 유리가구 계산
idx = 0
print("{} 가구의 종합계약 단일계약 청구서".format(calc.households[idx].name))
print("--------------------------------------")
print("종합계약 청구서 : {} 원".format(format(calc.households[idx].bill, ",")))
print("단일계약 청구서 : {} 원".format(format(single_calc.households[idx].bill, ",")))
print("--------------------------------------\n")

cnt = len(calc.households)
comp_cnt = 0
draw_cnt = 0
single_cnt = 0
for idx in range(0, cnt):
    if calc.households[idx].bill > single_calc.households[idx].bill:
        single_cnt += 1
    elif calc.households[idx].bill < single_calc.households[idx].bill:
        comp_cnt += 1
    else:
        draw_cnt

print("--------------------------------------")
print("전체 가구 : {} 가구".format(cnt))
print("종합계약 유리 : {} 가구".format(comp_cnt))
print("동일 : {} 가구".format(draw_cnt))
print("단일계약 유리 : {} 가구".format(single_cnt))
print("--------------------------------------\n")

입력값 정보------------------------------
공용부 Percentage : 30%
--------------------------------------

1월 요금 정보---------------------------
종합계약 아파트 전기 사용요금 : 19,271,430 원
단일계약 아파트 전기 사용요금 : 18,024,560 원
--------------------------------------

아파트1-101-1002 가구의 종합계약 단일계약 청구서
--------------------------------------
종합계약 청구서 : 47,470 원
단일계약 청구서 : 44,404 원
--------------------------------------

--------------------------------------
전체 가구 : 398 가구
종합계약 유리 : 105 가구
동일 : 0 가구
단일계약 유리 : 293 가구
--------------------------------------



## 5. 한 달 percentage ( 10% ~ 80% )

In [168]:
analysis_df = month_usage_df.set_index("month")
better_comp_df = pd.DataFrame()

for month in range(1, 13):
    # 1. 월별 사용량 데이터 파싱
    month_datas_df = pd.DataFrame(columns=["name","usage (kWh)"])
    for idx in analysis_df.loc[month].index:
        household_name = idx
        household_kWh = analysis_df.loc[month][idx]

        month_datas_df = month_datas_df.append({
            "name": household_name,
            "usage (kWh)": household_kWh
        }, ignore_index=True)

    # Thinking
    # 세대부 전기는 정해져 있는데,
    # 공용부 전기는 정해져 있지가 않아서 입력되는 percentage에 따라, 변화하도록

    # 다음과 같은 공식을 사용할 수 있다.
    # n -> 공용부가 전체 APT에서 차지할 percentage
    # APT : households_kWh = 100 : (100 - n)
    # APT : public_kWh = 100 : n

    # 이에 따라,
    # APT = (households_kWh * 100) / (100 - n)
    # public_kwh = APT - households_kwh
    min_per = 10
    max_per = 80
    rows = np.array([])

    for PUBLIC_PERCENTAGE in range(min_per, max_per + 1):
        households_kWh = sum(month_datas_df['usage (kWh)'].values)
        APT = round((households_kWh * 100) / (100 - PUBLIC_PERCENTAGE))
        public_kWh = round(APT - households_kWh)

        # 종합계약
        calc = ManagementOffice(
            month=month,
            peaks=peak_df, 
            households=month_datas_df,
            APT=APT,
            contract="종합계약",
            general_fee_info=['고압 A', 1]
        )

        # 단일계약
        single_calc = ManagementOffice(
            month=month,
            peaks=peak_df, 
            households=month_datas_df,
            APT=APT,
            contract="단일계약"
        )

        cnt = len(calc.households)
        comp_cnt = 0
        draw_cnt = 0
        single_cnt = 0
        for idx in range(0, cnt):
            if calc.households[idx].bill > single_calc.households[idx].bill:
                single_cnt += 1
            elif calc.households[idx].bill < single_calc.households[idx].bill:
                comp_cnt += 1
            else:
                draw_cnt

        rows = np.append(rows,comp_cnt)

        IPython.display.clear_output()

    better_comp_df = better_comp_df.append(
        pd.Series(rows, index=["{}%".format(_) for _ in range(min_per, max_per + 1)], name=month))
    better_comp_df

better_comp_df = better_comp_df.append(
    pd.Series(better_comp_df.mean(), name="mean")
)
better_comp_df

,10%,11%,12%,13%,14%,15%,16%,17%,18%,19%,...,71%,72%,73%,74%,75%,76%,77%,78%,79%,80%
1,11.000000,13.000000,15.000000,18.000000,18.000000,19.00,22.000000,24.000000,35.000000,44.00,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
2,13.000000,16.000000,18.000000,20.000000,21.000000,28.00,37.000000,41.000000,52.000000,60.00,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
3,52.000000,70.000000,82.000000,92.000000,92.000000,92.00,92.000000,92.000000,92.000000,92.00,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
4,65.000000,76.000000,93.000000,113.000000,113.000000,113.00,113.000000,113.000000,113.000000,113.00,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
6,4.000000,5.000000,5.000000,6.000000,8.000000,8.00,9.000000,12.000000,18.000000,19.00,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
9,62.000000,81.000000,95.000000,114.000000,137.000000,137.00,137.000000,137.000000,137.000000,137.00,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
10,57.000000,73.000000,88.000000,103.000000,115.000000,115.00,115.000000,115.000000,115.000000,115.00,...,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0,398.0
